# Mixed beverage data problems

This file outlines problems I've seen with the identification of 2017 [Mixed Beverage Gross Receipts](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php) files posted by on the Texas Comptroller's [data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/).

All through 2015 and 2016, the Mixed Beverage Tax Receipts have been posted using a specific naming convention that made sense in relation to their due date, quoting [online documentation](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php): the **"20th day of the month following the end of each calendar month (for example, April 20 for March activity)"**.

Typically, soon after the 20th, a file would be posted:
- The file name represented the month of its due date. For example, the receipts that were submitted for the April 20th, 2016 due date had a file name of MIXEDBEV_04_2016.CSV
- The contents of that file mostly contain receipts form the previous month. So, the April 2016 file would have mostly receipts from March 2016.

Beginning in 2017, the file posting to Texas Comptroller site has not followed this same format, making it confusing in relation to other data. As I write this on Aug. 1, 2017, the files that are posted are named neither for their due date, nor the contents.

## Why does it matter?

There is very little information about this data beyond a boilerplate file description that is the same for every month and a sparse [file layout](https://comptroller.texas.gov/auto-data/odc/MIXEDBEVTAX_LYOT.txt). This makes the title of the record and name of the file the only way the public can distinguish the record beyond the contents of the file itself. Open data as monthly government reports like this are often accessed through programming and APIs, and when basic naming conventions are not followed it can lead to errors and confusion for the public.

### Example of historical use

You can review how these files are named by viewing old one's by visiting the [data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/) using a Chrome or Firefox browser and rolling your cursor over the CSV link to see the linked filename in the message window at the bottom of the browser.

![2017_data](../data_problem/dec2015_example.png)

All the 2015 and 2016 files on the site follow the same pattern: The title of the file matches the month/year of the file name, which is the month of the due date of the data.

### Example of 2017 inconsistency

With all the 2017 data, the title of the data has been changed to two months prior, when that date has nothing to do with the data. This file below was posted to the data portal on July 31, 2017, then renamed as 'MAY 2017' file. There is little reason to mark this file as May since the data is mostly June and the filing deadline was in July. It is confusing and inconsistent with a more relevant naming convention adhered to for two years in 2015 and 2016.

![2017_data](../data_problem/july2017_example.png)

## Recommendation and request

Please return to the original title convention that matches the month the data is updated. Add records for the true January and February 2017 files, which are no longer listed on the portal.

## Programmatic look into Mixed Beverage Receipts file contents

I suggest above to return to that the orginal file naming convention used in 2015 and 2016 because it references the due date of the reports, as outlined [by Comptroller reference documentation](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php).

Because that due date is in the middle of the month, the contents of files uploaded from that due date most often reflect the prior month. Using at title three months prior to the due date, as is done with the 2017 files, is confusing and doesn't relate the actual data.

In case interested parties want to see how the file names reflect the content within the files, here is a transparent, repeatable analysis of the file titled 'DEC 2016' and 'JAN 2017' to illustrate the differences.

### File setup

The next several frames include code necessary to process the files, using an open source Python data package called [agate](http://agate.readthedocs.io/). Skip down to **Mixed bev file analysis** to get to the results.

In [1]:
# imports the libraries we will use
import agate
from decimal import Decimal
import re

# this surpresses a timezone warning
import warnings
warnings.filterwarnings('ignore')

# sets the column names of the original data set.
column_names = [
    'TABC Permit Number',
    'Trade Name',
    'Location Address',
    'Location City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Blank',
    'Report Period',
    'Report Tax'
]
# Helps us import some text fields that may be considered numbers in error.
specified_types = {
    'Location Zip Code': agate.Text(),
    'Location County Code': agate.Text()
}

### MIXBEV IMPORT AND PRINT FUNCTION
# This function imports the downloade file and then
# prints information about the Report Period.
# Details are outlined in comments below
###
def process_mixbev(title, path):

    # splits the path to get the file name
    file_name = path.rsplit('/', 1)[-1]

    # creates file location to downloaded data
    file_location = '../data_problem/' + file_name
    
    # this imports the file specified above, along with the proper types
    mixbev_raw = agate.Table.from_csv(
        file_location, column_names,
        encoding='iso-8859-1',
        column_types=specified_types
    )

    # Pivot the mixbev table by Report Period.
    # We then order the table by Count in descending order
    mixbev_by_period = mixbev_raw.pivot('Report Period').order_by('Count', reverse=True)

    # set up print statement
    print('The entry titled:\n {}\n\n... has the file name of:\n {}\n'.format(
            title,
            file_name
        ))

    print('The count of records by Report Period is:\n')
    
    # prints the table of period and number of records
    mixbev_by_period.limit(5).print_table(max_rows=None)
    
    return(mixbev_raw)

### Mixed bev file analysis

To analyze these, I'm feeding the title and file path to a function that prints the number of records for each Report Period in the data. While I do reference the full path to the data online, I previously downloaded the files for speed and source control.

- **`mon_year_title`** is the title used for that file on the comptroller's website
- **`mon_year_file_path`** is the url to the file on the comptroller's website

### DEC 2016

This first data set was titled **Mixed Beverage Tax Receipts - DEC 2016** and had a file name of `MIXEDBEV_12_2016.CSV`, which is representative of how the filenames, data and titles were typically set throughout 2015 and 2016. Most of the reports were from the previous month, November 2016.

In [2]:
# Mixed Beverage Tax Receipts - DEC 2016
dec_2016_title = 'Mixed Beverage Tax Receipts - DEC 2016'
dec_2016_file_path = 'https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_12_2016.CSV'

dec_mixbev = process_mixbev(dec_2016_title, dec_2016_file_path)

The entry titled:
 Mixed Beverage Tax Receipts - DEC 2016

... has the file name of:
 MIXEDBEV_12_2016.CSV

The count of records by Report Period is:

| Report Period |  Count |
| ------------- | ------ |
| 2016/11       | 14,224 |
| 2016/10       |  1,624 |
| 2016/09       |    158 |
| 2016/08       |     48 |
| 2016/12       |     41 |


### JAN 2017

The second example is the file marked for January 2017, which has a filename `MIXEDBEV_03_2017.CSV` which is the March due date. The real data files for due dates in January and February 2017 are not available through the portal search, which is another inconsistency with this change.

The result is this file for JAN 2017 is really about data for the FOLLOWING month, February. The due date for these reports was March 20th, 2017, and the file was actually uploaded in March of 2017, despite the January title.

In [3]:
# Mixed Beverage Tax Receipts - JAN 2017
jan_2017_title = 'Mixed Beverage Tax Receipts - JAN 2017'
jan_2017_file_path = 'https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_03_2017.CSV'

jan_2017_mixbev = process_mixbev(jan_2017_title, jan_2017_file_path)

The entry titled:
 Mixed Beverage Tax Receipts - JAN 2017

... has the file name of:
 MIXEDBEV_03_2017.CSV

The count of records by Report Period is:

| Report Period |  Count |
| ------------- | ------ |
| 2017/02       | 14,090 |
| 2017/01       |  1,423 |
| 2016/12       |    141 |
| 2016/11       |     52 |
| 2017/03       |     32 |


### 2017 "change" continues

Just to show this wasn't a one-month change, here are the "FEB 2017" and "MAR 2017" files.

In [4]:
# Mixed Beverage Tax Receipts - FEB 2017
feb_2017_title = 'Mixed Beverage Tax Receipts - FEB 2017'
feb_2017_file_path = 'https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_04_2017.CSV'

feb_2017_mixbev = process_mixbev(feb_2017_title, feb_2017_file_path)

The entry titled:
 Mixed Beverage Tax Receipts - FEB 2017

... has the file name of:
 MIXEDBEV_04_2017.CSV

The count of records by Report Period is:

| Report Period |  Count |
| ------------- | ------ |
| 2017/03       | 14,144 |
| 2017/02       |  1,718 |
| 2017/01       |    258 |
| 2016/12       |     61 |
| 2017/04       |     49 |


In [5]:
# Mixed Beverage Tax Receipts - MAR 2017
mar_2017_title = 'Mixed Beverage Tax Receipts - MAR 2017'
mar_2017_file_path = 'https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_05_2017.CSV'

mar_2017_mixbev = process_mixbev(mar_2017_title, mar_2017_file_path)

The entry titled:
 Mixed Beverage Tax Receipts - MAR 2017

... has the file name of:
 MIXEDBEV_05_2017.CSV

The count of records by Report Period is:

| Report Period |  Count |
| ------------- | ------ |
| 2017/04       | 14,410 |
| 2017/03       |  1,819 |
| 2017/02       |    187 |
| 2017/01       |     52 |
| 2017/05       |     35 |


## Postscript: File posted correctly, then renamed later

Not to sound like a conspiracy theorist, but the July 2017 due date file `MIXEDBEV_07_2017.CSV` was originally posted to the site "correctly" on July 31, 217, then renamed later in the day to the inconsistent method used in 2017. I know this because we [open source software](https://newsklaxon.org/) to watch and alert us when the data portal is updated so we know when new data is available. Green lines are additions, and red lines are deletions.

First update for the file `MIXEDBEV_07_2017.CSV` with correct name:

![2017_data](../data_problem/july31_v1.png)

And then our alert after the file was renamed to May half an hour later.

![2017_data](../data_problem/july31_v2.png)

I've seen that pattern more than once.
